# CDRC

In [ ]:
import os
import re
import zipfile
from pathlib import Path

import duckdb
import fiona
import pandas as pd
from PyPDF2 import PdfReader

from cdrc import cdrc_get_metadata, cdrc_login
from utils import download_file, get_filesize_mb


cdrc = Path("data/cdrc")
cdrc.mkdir(parents=True, exist_ok=True)

cdrc_session = cdrc_login(
    os.environ.get("CDRC_USERNAME"), os.environ.get("CDRC_PASSWORD")
)
if cdrc_session is None:
    raise "Check CDRC Login / Session Request"

con = duckdb.connect()
for extension in ("spatial", "parquet", "httpfs"):
    con.install_extension(extension)
    con.load_extension(extension)

In [ ]:
# for writeup
# https://www.ons.gov.uk/methodology/geography/geographicalproducts/areaclassifications/2021residentialbasedareaclassifications

### [Output Area Classification - 2021](https://data.cdrc.ac.uk/dataset/output-area-classification-2021)

In [ ]:
source_url = "https://data.cdrc.ac.uk/dataset/output-area-classification-2021"
data_url = "https://data.cdrc.ac.uk/system/files/OAC_gpkg.zip"
pdf_url = "https://data.cdrc.ac.uk/system/files/Pen_Portraits_EW.pdf"

dataset = "oac_2021"
tmp_file = Path(data_url.split("/")[-1].lower())

data_file = cdrc.joinpath(f"{dataset}.parquet.gzip")
metadata_file = cdrc.joinpath(f"{dataset}_metadata.parquet")
pdf_file = cdrc.joinpath(f"{dataset}.pdf")
lookup_file = cdrc.joinpath(f"{dataset}_lookup.parquet")

#### Output Area Classification - 2021

In [ ]:
success = download_file(data_url, tmp_file, cdrc_session)
if not success:
    raise f"Error Downloading Data: {data_url}

In [ ]:
gpkg = Path("OAC.gpkg")
with zipfile.ZipFile(tmp_file, "r") as zip_ref:
    zip_ref.extract(gpkg.name)
fiona.listlayers(gpkg.name)

In [ ]:
layer = "OAC_SuperGroup"

query = f"""
COPY (
  SELECT
    OA21CD AS oa_code_21 -- primary key
    , la23cd AS la_code_23
    , sg::INT::TEXT AS super_grp
    , g AS grp
    , subg AS sub_grp
    , ST_ASWKB(geom) AS geom_wkb_4326
  FROM ST_Read('{gpkg}', layer='{layer}')

  -- ordered to enable better gzip compression
  ORDER BY 2, ST_XMIN(geom)
)
TO '{data_file}'
(FORMAT 'PARQUET', CODEC 'GZIP')
;
"""

con.execute(query)

In [ ]:
print(f"Size of '{tmp_file}': {get_filesize_mb(tmp_file):.2f} MB")
print(f"Size of '{gpkg}': {get_filesize_mb(gpkg):.2f} MB")
print(f"Size of '{data_file}': {get_filesize_mb(data_file):.2f} MB")

In [ ]:
# remove temp files
tmp_file.unlink()
gpkg.unlink()

#### Output Area Classification - 2021 Metadata

In [ ]:
metadata = cdrc_get_metadata(source_url, cdrc_session)

if metadata is None:
    raise "Error Getting Metadata"

metadata.to_parquet(metadata_file)
metadata

#### Output Area Classification - 2021 PDF & OAC Lookups

In [ ]:
success = download_file(pdf_url, pdf_file, cdrc_session)
if not success:
    raise f"Error Downloading Data: {data_url}"

In [ ]:
# get OAC (Super/Sub)Group description pages (from page 12 to the end)
reader = PdfReader(pdf_file)
text = "".join(page.extract_text() for page in reader.pages[11:])

In [ ]:
# regex patterns
supergroup_desc_pattern = r"\n(\d): (.*?)(?=\n\d:|\n\d[a-z]:|\Z)"
group_desc_pattern = r"(\d[a-z]): (.*?)(?=\n\d:|\n\d[a-z]:|\n\d[a-z]\d:|\Z)"
subgroup_desc_pattern = r"(\d[a-z]\d)[: ](.*?)(?=Examples|England)"

# extract
supergroup_descriptions = re.findall(supergroup_desc_pattern, text, re.DOTALL)
group_descriptions = re.findall(group_desc_pattern, text, re.DOTALL)
subgroup_descriptions = re.findall(subgroup_desc_pattern, text, re.DOTALL)

In [ ]:
# convert to dataframes / cleanup
supergroups_df = pd.DataFrame(
    supergroup_descriptions, columns=["super_grp", "super_grp_desc"]
).assign(
    super_grp_name=lambda df: df["super_grp_desc"].str.split("\n").str[0].str.strip(),
    super_grp_desc=lambda df: df["super_grp_desc"].str.strip(),
)

groups_df = pd.DataFrame(group_descriptions, columns=["grp", "grp_desc"]).assign(
    grp_name=lambda df: df["grp_desc"].str.split("\n").str[0].str.strip(),
    super_grp=lambda df: df["grp"].str.extract(r"(\d)"),
    grp_desc=lambda df: df["grp_desc"].str.strip(),
)

subgroups_df = pd.DataFrame(
    subgroup_descriptions, columns=["sub_grp", "sub_grp_desc"]
).assign(
    sub_grp_name=lambda df: df["sub_grp_desc"].str.split("\n").str[0].str.strip(),
    super_grp=lambda df: df["sub_grp"].str.extract(r"(\d)"),
    grp=lambda df: df["sub_grp"].str.extract(r"(\d[a-z])"),
    sub_grp_desc=lambda df: df["sub_grp_desc"].str.strip(),
)

In [ ]:
# The 2021 OAC classification comprises 8 Supergroups, 21 Groups and 52 Subgroups
assert supergroups_df.shape[0] == 8
assert groups_df.shape[0] == 21
assert subgroups_df.shape[0] == 52

In [ ]:
columns = [
    "sub_grp",
    "grp",
    "super_grp",
    "sub_grp_name",
    "grp_name",
    "super_grp_name",
    "sub_grp_desc",
    "grp_desc",
    "super_grp_desc",
]
subgroup_flat_df = subgroups_df.merge(
    groups_df, on=["super_grp", "grp"], how="left"
).merge(supergroups_df, on=["super_grp"], how="left")[columns]

subgroup_flat_df.head(1)

In [ ]:
subgroup_flat_df.to_parquet(lookup_file)

### [CDRC Residential Mobility Index](https://data.cdrc.ac.uk/dataset/cdrc-residential-mobility-index)

Availale at LSOA and LAD, for initial work use higher level LAD

For Geometries:
 - https://geoportal.statistics.gov.uk/datasets/ons::local-authority-districts-may-2023-boundaries-uk-bfc/explore?location=52.653485%2C-0.681218%2C6.50

In [ ]:
source_url = "https://data.cdrc.ac.uk/dataset/cdrc-residential-mobility-index"
data_url = "https://data.cdrc.ac.uk/system/files/hh_churn_oslaua_2023.csv"
pdf_url = (
    "https://data.cdrc.ac.uk/system/files/CDRC%20Residential%20Mobility%20Index.pdf"
)

dataset = "residential_mobility_lad_2023"
tmp_file = Path(data_url.split("/")[-1].lower())

data_file = cdrc.joinpath(f"{dataset}.parquet.gzip")
metadata_file = cdrc.joinpath(f"{dataset}_metadata.parquet")
pdf_file = cdrc.joinpath("residential_mobility_2023.pdf")

#### Residential Mobility (LAD 2011-2023)

In [ ]:
success = download_file(data_url, tmp_file, cdrc_session)
if not success:
    raise f"Error Downloading Data: {data_url}"

In [ ]:
query = f"""
COPY( SELECT * FROM READ_CSV('{tmp_file}', auto_detect=TRUE) )
TO '{data_file}'
(FORMAT 'PARQUET', CODEC 'GZIP')
;
"""

con.execute(query)

#### Residential Mobility (LAD 2011-2023) - Metadata

In [ ]:
metadata = cdrc_get_metadata(source_url, cdrc_session)
if metadata is None:
    raise "Error Getting Metadata"

metadata.to_parquet(metadata_file)
metadata

#### Residential Mobility (LAD 2011-2023) - PDF

In [ ]:
success = download_file(pdf_url, pdf_file, cdrc_session)
if not success:
    raise f"Error Downloading Data: {pdf_url}"

In [ ]:
# remove temp files
tmp_file.unlink()